In [1]:
from unet import unet_constructor as GUnet
import dataloader as dataloader
from loss import dice_loss, cross_entropy_loss, random_cross_entropy
import transforms as t
import torch.nn.functional as F
import torch
import torch.nn as nn
import torchvision.transforms as tt
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import matplotlib.pyplot as plt
import pickle
import skimage.io as io
import os

writer = SummaryWriter()

In [2]:
data = dataloader.stack(path='./Data/train',
                        joint_transforms=[t.to_float(),
                                          t.reshape(),
                                          t.nul_crop(),
                                          t.random_crop([300, 300, 19]),
                                          t.random_rotate(),
                                          t.random_affine()
                                          ],
                        image_transforms=[
                                          t.drop_channel(1), 
                                          t.random_gamma((.8,1.2)),
                                          t.spekle(),
                                          t.normalize([0.5, 0.5, 0.5, 0.5], [0.5, 0.5, 0.5, 0.5])
                                          ]
                        )

val_data = dataloader.stack(path='./Data/train',
                        joint_transforms=[t.to_float(),
                                          t.reshape(),
                                          t.random_crop([512, 512, 19]),
                                          t.random_rotate(90),
                                          #t.random_affine
                                          ],
                        image_transforms=[
                                          #t.random_gamma((.8,1.2)),
                                          #t.spekle(),
                                          t.normalize([0.5, 0.5, 0.5, 0.5], [0.5, 0.5, 0.5, 0.5])
                                          ]
                        )



In [3]:
device = 'cuda:0'

test = GUnet(image_dimensions=3,
             in_channels=4,
             out_channels=1,
             feature_sizes=[16,32,64,128],
             kernel={'conv1': (3, 3, 2), 'conv2': (3, 3, 1)},
             upsample_kernel=(8, 8, 2),
             max_pool_kernel=(2, 2, 1),
             upsample_stride=(2, 2, 1),
             dilation=1,
             groups=2).to(device)


test = test.type(torch.float)

image, mask, pwl = data[0]

out = test.forward(image.float().to(device))
out_loss = dice_loss(out, mask.to(device))#, pwl.float().to('cuda'))



In [4]:
test.load('Apr21_chris-MS-7C37_2.unet')
test.cuda()
test.train()

unet_constructor(
  (out_conv): Conv3d(16, 1, kernel_size=(1, 1, 1), stride=(1, 1, 1))
  (down_steps): ModuleList(
    (0): Down(
      (conv1): Conv3d(4, 16, kernel_size=(3, 3, 2), stride=(1, 1, 1), groups=2)
      (conv2): Conv3d(16, 16, kernel_size=(3, 3, 1), stride=(1, 1, 1), groups=2)
      (batch1): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (batch2): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (1): Down(
      (conv1): Conv3d(16, 32, kernel_size=(3, 3, 2), stride=(1, 1, 1), groups=2)
      (conv2): Conv3d(32, 32, kernel_size=(3, 3, 1), stride=(1, 1, 1), groups=2)
      (batch1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (batch2): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (2): Down(
      (conv1): Conv3d(32, 64, kernel_size=(3, 3, 2), stride=(1, 1, 1), groups=2

In [ ]:
epoch = 0 
k = 0
running_loss = 0
device = 'cuda:0'

losses_dice=[]
avg_losses_dice = []
losses_BCE=[]
avg_losses_BCE = []



In [ ]:
lr = .01
gamma = .75

optimizer = torch.optim.Adam(test.parameters(), lr = lr)
#optimizer = torch.optim.SGD(test.parameters(), lr=lr, momentum=.99, nesterov=True)

scheduler_1 = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=gamma)
scheduler_2 = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=100)


In [ ]:
while True:  # loop over the dataset multiple times
    epoch_loss_dice=[]
    epoch_loss_BCE=[]
    epoch_loss_joint=[]
    for i in range(len(data)):
        image, mask, pwl  = data[i]

        optimizer.zero_grad()
        out = test(image.float().to('cuda'))


       #out_loss = cross_entropy_loss(out, mask.to('cuda'), pwl.to('cuda'), 5) +\
        out_loss=  dice_loss(out, mask.to('cuda'))
        epoch_loss_joint.append(out_loss.item())
        
        print(f'\r\033[32mEpoch: \033[0m{epoch} |',end='')
        print(f' JOINT |', end='')
        print(f' {i} |',end='')
        print(f' \033[35mPOL: \033[0m{str(running_loss)[0:8]} -->', end='')
        print(f' \033[35mOL: \033[0m{str(out_loss.item())[0:8]}', end='')

        out_loss.backward()
        optimizer.step()

        running_loss = out_loss.item()

        if epoch+1 % 150 == 0:
            scheduler_1.step() 

    
    to_tb = np.array(epoch_loss_joint).sum()/len(epoch_loss_joint)
    writer.add_scalar('Dice Loss Train', to_tb, epoch)
    epoch += 1




In [ ]:
epoch_loss_dice

In [ ]:
test.save('Apr27_chris-MS-7C37_2.unet')

In [ ]:
plt.semilogy(avg_losses_dice[10::])

In [5]:
image, mask, pwl  = val_data[8]
test.eval()
#test.cpu()
with torch.no_grad(): 
    out = test(image.float().cuda())


RuntimeError: CUDA out of memory. Tried to allocate 184.00 MiB (GPU 0; 10.76 GiB total capacity; 4.62 GiB already allocated; 101.00 MiB free; 4.90 GiB reserved in total by PyTorch)

In [ ]:
pred = F.sigmoid(out) > .5
for i in range(14):
    #plt.imshow(image[0,0:3,:,:,i].transpose(0,1).transpose(1,2)*.5 + .5)
    plt.imshow(image[0,0:3,:,:,i].transpose(0,1).transpose(1,2)*.5 + .5)
    #plt.imshow(mask[0,0,:,:,i], cmap = 'Greens')
    plt.imshow(pred[0,0,:,:,i].cpu().detach().numpy(), cmap=plt.cm.nipy_spectral,alpha=.2)
    plt.show()


In [ ]:
pred.min()

In [ ]:
io.imsave('overlap.tif', pred.cpu().squeeze(0).transpose(0,3).float().numpy())

In [ ]:
n.shape

In [ ]:
image.min()